In [1]:
pip install Flask Flask-SocketIO


                                              0.0/57.0 kB ? eta -:--:--
     ---------------------------------------- 57.0/57.0 kB 1.5 MB/s eta 0:00:00
                                              0.0/53.5 kB ? eta -:--:--
     ---------------------------------------- 53.5/53.5 kB 2.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
video_conferencing_website/
│
├── static/
│   └── script.js
│
├── templates/
│   └── index.html
│
└── app.py


In [ ]:
from flask import Flask, render_template
from flask_socketio import SocketIO

app = Flask(__name__)
socketio = SocketIO(app)

@app.route('/')
def index():
    return render_template('index.html')

if __name__ == '__main__':
    socketio.run(app, debug=True)


In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>Video Conferencing</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.1.3/socket.io.js"></script>
    <script src="{{ url_for('static', filename='script.js') }}"></script>
</head>
<body>
    <h1>Video Conferencing</h1>
    <div id="local-video"></div>
    <div id="remote-video"></div>
    <button id="start-button">Start Call</button>
    <button id="end-button">End Call</button>
</body>
</html>


In [ ]:
document.addEventListener('DOMContentLoaded', () => {
    const localVideo = document.getElementById('local-video');
    const remoteVideo = document.getElementById('remote-video');
    const startButton = document.getElementById('start-button');
    const endButton = document.getElementById('end-button');

    const socket = io.connect('http://' + document.domain + ':' + location.port);

    let localStream;
    let remoteStream;

    const constraints = {
        video: true,
        audio: true,
    };

    startButton.addEventListener('click', async () => {
        try {
            localStream = await navigator.mediaDevices.getUserMedia(constraints);
            localVideo.srcObject = localStream;

            socket.emit('start_call');
        } catch (error) {
            console.error('Error accessing media devices:', error);
        }
    });

    endButton.addEventListener('click', () => {
        if (localStream) {
            localStream.getTracks().forEach(track => track.stop());
            localVideo.srcObject = null;

            socket.emit('end_call');
        }
    });

    socket.on('receive_call', () => {
        remoteStream = new MediaStream();
        remoteVideo.srcObject = remoteStream;

        localStream.getTracks().forEach(track => remoteStream.addTrack(track));

        socket.emit('accept_call');
    });

    socket.on('end_call', () => {
        if (remoteStream) {
            remoteStream.getTracks().forEach(track => track.stop());
            remoteVideo.srcObject = null;
        }
    });
});


In [ ]:
python app.py
